In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("./mnist/data", one_hot= True)

#옵션 설정
total_epoch = 100
batch_size =100
learning_rate = 0.02


W0724 06:36:41.360783 140081129940864 deprecation.py:323] From <ipython-input-3-15e6802bad66>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0724 06:36:41.364447 140081129940864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0724 06:36:41.367087 140081129940864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0724 06:36:41.811398 140081129940864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0724 06:36:41.813281 140081129940864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0724 06:36:41.974078 140081129940864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
#신명강 레이어 구성 옵션
n_hidden = 256
n_input = 28*28
n_noise = 128 #생성기의 입력값으로 사용할 노이즈의 크기

In [0]:
#신경망 모델 구성
#GAN도 Unsupervised 학습이므로 Autoencoder 처럼 Y를 사용 안함
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise]) #노이즈 Z를 입력값으로 사용

#생성기 신경망에 사용하는 변수들
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

#판별기 신경망에 사용하는 변수들
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
#판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라 값
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

#생성기(G) 신경망을 구성
def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2)+G_b2)
  return output

#판별기(D) 신경망을 구성
def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1)+ D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2)+ D_b2)
  return output

#랜덤한 노이즈(Z)
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))


In [0]:
#노이즈를 이용해 랜덤한 이미지를 생성합니다
G = generator(Z)
#노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다
D_gene = discriminator(G)

#진짜 이미지를 이용해 판별한 값을
D_real = discriminator(X)

In [0]:
loss_D = tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))

loss_G = tf.reduce_mean(tf.log(D_gene))

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

#최적화 하려는 loss_D와 loss_G에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    noise = get_noise(batch_size, n_noise)
    
    #판별기와 생성기 신경망을 각각 학습시킵니다
    _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X:batch_xs, Z:noise})
    _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Z:noise})
    
  print('Epoch:', '%04d'% epoch,'D loss:{:4}'.format(loss_val_D), 'G loss:{:4}'.format(loss_val_G))
  
  #학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
  if epoch ==0 or (epoch + 1) % 10 ==0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict = {Z: noise})
    
    fig, ax = plt.subplots(1, sample_size, figsize = (sample_size, 1))
    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i],(28,28)))
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)
    print("최적화 완료!")

Epoch: 0000 D loss: 0.0 G loss:-inf
최적화 완료!
Epoch: 0001 D loss: 0.0 G loss:-inf
Epoch: 0002 D loss: 0.0 G loss:-inf
Epoch: 0003 D loss: 0.0 G loss:-inf
Epoch: 0004 D loss: 0.0 G loss:-inf
Epoch: 0005 D loss: 0.0 G loss:-inf
Epoch: 0006 D loss: 0.0 G loss:-inf
Epoch: 0007 D loss: 0.0 G loss:-inf
Epoch: 0008 D loss: 0.0 G loss:-inf
Epoch: 0009 D loss: 0.0 G loss:-inf
최적화 완료!
Epoch: 0010 D loss: 0.0 G loss:-inf
Epoch: 0011 D loss: 0.0 G loss:-inf
Epoch: 0012 D loss: 0.0 G loss:-inf
Epoch: 0013 D loss: 0.0 G loss:-inf
Epoch: 0014 D loss: 0.0 G loss:-inf
Epoch: 0015 D loss: 0.0 G loss:-inf
Epoch: 0016 D loss: 0.0 G loss:-inf
Epoch: 0017 D loss: 0.0 G loss:-inf
Epoch: 0018 D loss: 0.0 G loss:-inf
Epoch: 0019 D loss: 0.0 G loss:-inf
최적화 완료!
Epoch: 0020 D loss: 0.0 G loss:-inf
Epoch: 0021 D loss: 0.0 G loss:-inf
Epoch: 0022 D loss: 0.0 G loss:-inf
Epoch: 0023 D loss: 0.0 G loss:-inf
Epoch: 0024 D loss: 0.0 G loss:-inf
Epoch: 0025 D loss: 0.0 G loss:-inf
Epoch: 0026 D loss: 0.0 G loss:-inf
Epoc

In [0]:
!mkdir -p samples